In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.neighbors import NearestNeighbors
import lightgbm as lightgbm
import numpy as np
import pandas as pd
import pickle
from sklearn.datasets import load_svmlight_file

def read_data(path):
    if path.endswith(".svm"):
        data = load_svmlight_file(path, query_id=True)
        return data[0], data[1], data[2]
    elif path.endswith(".jsonl"):
        data = pd.read_json(path_or_buf=path, lines=True)
        return data
    elif path.endswith(".txt"):
        data = pd.read_csv(path, sep=" ")
        return data
    else:
        print("Invalid path")
        return None

In [ ]:
with open('/data/Soheil-data/Python Projects/KNN-LETOR/train_svm_features_avg.pkl', 'rb') as f:
    preprocessed_train_cum_features = pickle.load(f) #main dict train with qids and features

In [ ]:
with open('/data/Soheil-data/Python Projects/KNN-LETOR/test_svm_features_avg.pkl', 'rb') as f:
    preprocessed_test_cum_features = pickle.load(f) #main dict with test qids and features

In [ ]:
train_svm_directory = "/data/Sepehr-data/istella22/train.svm"
train_svm_features, train_svm_relevance, train_svm_q_id = read_data(train_svm_directory)


In [ ]:
arr_train_svm_features =  train_svm_features.toarray()
df_train_svm_features = pd.DataFrame({"features": [row for row in arr_train_svm_features], "q_id": train_svm_q_id, "relevance": train_svm_relevance})

In [ ]:
df_train_svm_features['q_id'][0:5]

In [ ]:
with open('/data/Soheil-data/Python Projects/KNN-LETOR/test_svm_features_rel.pkl', 'rb') as f:
    test_data_all = pickle.load(f)


In [ ]:
preprocessed_train_cum_features= pd.DataFrame(preprocessed_train_cum_features)
preprocessed_test_cum_features= pd.DataFrame(preprocessed_test_cum_features)
train_data_all= df_train_svm_features#pd.DataFrame(train_data_all)
test_data_all= pd.DataFrame(test_data_all)

In [ ]:
train_svm_features = []
train_svm_relevance = []
train_svm_q_id =[]
qid_avg_train = preprocessed_train_cum_features["q_id"].tolist()
features_avg_train = preprocessed_train_cum_features["features"].tolist()
qid_avg_train = np.vstack(qid_avg_train)
features_avg_train = np.vstack(features_avg_train)
preprocessed_train_cum_features = []

In [ ]:
qid_avg_test = preprocessed_test_cum_features["q_id"].tolist()
features_avg_test = preprocessed_test_cum_features["features"].tolist()
qid_avg_test = np.vstack(qid_avg_test)
features_avg_test = np.vstack(features_avg_test)
preprocessed_test_cum_features = []


In [ ]:
qid_all_train= train_data_all["q_id"].tolist()

relevance_all_train = train_data_all["relevance"].tolist()
features_all_train = train_data_all["features"].tolist()
qid_all_train = np.vstack(qid_all_train)
relevance_all_train = np.vstack(relevance_all_train)
features_all_train = np.vstack(features_all_train)
train_data_all = []

In [ ]:
train_data_all = []

qid_all_test = test_data_all["q_id"].tolist()
relevance_all_test = test_data_all["relevance"].tolist()
features_all_test = test_data_all["features"].tolist()
qid_all_test = np.vstack(qid_all_test)
relevance_all_test = np.vstack(relevance_all_test)
features_all_test = np.vstack(features_all_test)
test_data_all= []

In [ ]:
# Get unique elements and their counts
unique_elements, counts = np.unique(qid_all_test, return_counts=True)

# Print the counts for each unique element
for element, count in zip(unique_elements, counts):
    if(count == 1):
        print(f"Element {element} appears {count} times.")

In [ ]:
import time
import warnings
warnings.filterwarnings("ignore")
#find the nearest neighbour
#key2: features
#key1: qids
def get_nearest_neighbour(test_q, features_avg_train,k):
    knn = NearestNeighbors(n_neighbors=k)
    features_avg_train = np.array(features_avg_train)
    
    knn.fit(features_avg_train)
    test_q = test_q.reshape((1, -1))
    dist_loc = knn.kneighbors(test_q)
    dist = dist_loc[0]
    loc = dist_loc[1]
    return dist, loc
#these have relevance score, qids, and features
#key_0 = relevance score, key_qids, key2_features list
"""
test_unique_qids = []    # a list for getting the qids of tests
for d in qid_avg_test:
    if d not in test_unique_qids:
        test_unique_qids.append(d)
"""
k_n= 300
#qids_test = []
ncdg_cum = []
for i in range(0,len(qid_avg_test)):
   
    #get features corresponding to the qids of the test
    #qids_test.append(qid)
    #test_q = #[d['features'] for d in preprocessed_train_cum_features if d.get('q_id') == key1_value]
    #get the nearest neighbour of the test_q
    test_feature = features_avg_test[i]
    
    features_avg_test = features_avg_test.reshape((features_avg_test.shape[0],-1))
    
    features_avg_train = features_avg_train.reshape((features_avg_train.shape[0], 220))
    
    dist , loc = get_nearest_neighbour(test_feature, features_avg_train,k_n)
    print("KNN done...")
    #get the values of the second key(features) of the trainset
    selected_train_set = features_avg_train[loc]
    #get the nearest query ids and add them to this list
    matching_train_qids = qid_avg_train[loc]
    matching_train_qids = matching_train_qids.reshape(-1,1)
    new_train_features = []
    new_train_qids = []
    new_train_relevances = []
   
    #for qid in matching_train_qids:  
    #indices = np.where(np.isin(qid_all_train, matching_train_qids))[0]
    new_train_qids.append(qid_all_train[np.where(np.isin(qid_all_train, matching_train_qids))[0]])
    new_train_features.append(features_all_train[np.where(np.isin(qid_all_train, matching_train_qids))[0]])
    new_train_relevances.append(relevance_all_train[np.where(np.isin(qid_all_train, matching_train_qids))[0]])
    
    """
    new_train_qids.append(qid_all_train[np.where(qid_all_train == matching_train_qids)[0]])
    new_train_features.append(features_all_train[np.where(qid_all_train == matching_train_qids)[0]])
    new_train_relevances.append(relevance_all_train[np.where(qid_all_train == matching_train_qids)[0]])
    """
    """
    for j in range(0, len(train_data_ids)):
        new_train_qids.append(qid_all_train[train_data_ids[j]])
        new_train_features.append(features_all_train[train_data_ids[j]])
        new_train_relevances.append(relevance_all_train[train_data_ids[j]])
    """
    
    new_train_relevances = np.array(new_train_relevances)
    new_train_features = np.array(new_train_features[0])
    new_train_qids = np.array(new_train_qids)

    # Reshape "relevance" and "qid" arrays
    new_train_relevances = new_train_relevances.reshape(-1)
    new_train_qids = new_train_qids.reshape(-1)
    #new_train_features = new_train_features.tolist()
    # Concatenate arrays and list
    #train_data = np.column_stack((new_train_relevances, new_train_qids, new_train_features))
    data_train_tmp = {
    'relevance': new_train_relevances,
    'qid': new_train_qids,
    'features': new_train_features.tolist()  # Convert features to a list of lists
    }
    
    # Create a DataFrame
    train_group = pd.DataFrame(data_train_tmp)
    
    test_data_ids = np.where(qid_all_test == qid_avg_test[i])[0]
    new_test_qids = []
    new_test_features = []
    new_test_relevances = []
    
    for j in range(0, len(test_data_ids)):
            new_test_qids.append(qid_all_test[test_data_ids[j]])
            new_test_features.append(features_all_test[test_data_ids[j]])
            new_test_relevances.append(relevance_all_test[test_data_ids[j]])
    
    new_test_qids = np.array(new_test_qids)
    new_test_features = np.array(new_test_features)
    new_test_relevances = np.array(new_test_relevances)
    
    new_test_relevances = new_test_relevances.reshape(-1)
    new_test_qids = new_test_qids.reshape(-1)
    
    data_test_tmp = {
    'relevance': new_test_relevances,
    'qid': new_test_qids,
    'features': new_test_features.tolist()  # Convert features to a list of lists
    }
    test_group = pd.DataFrame(data_test_tmp)
    
    qids_train = train_group.groupby("qid")["qid"].count().to_numpy()
    qids_test = test_group.groupby("qid")["qid"].count().to_numpy()
    
    new_train_relevances = new_train_relevances.astype(int)
    new_test_relevances = new_test_relevances.astype(int)
    a = time.time()
    ranker = lightgbm.LGBMRanker(
                        objective="lambdarank",
                        boosting_type = "gbdt",
                        #n_estimators = 1,
                        importance_type = "gain",
                        force_col_wise=True,
                        metric= "ndcg",
                        #num_leaves = 250,
                        #learning_rate = 0.05,
                        max_depth = -1,
                        #min_data_in_bin=10, 
                        #max_bin=250,
                        #device ='gpu',
                        label_gain =[i for i in range(max(new_train_relevances.max(), new_test_relevances.max()) + 1)])
    
    # Training the model
    ranker.fit(
          X=new_train_features,
          y=new_train_relevances,
          group=qids_train)
          #eval_set=[(new_train_features, new_train_relevances),(new_test_features, new_test_relevances)],
          #eval_group=[qids_train, qids_test],
          #eval_at=[10])
    
    test_pred = ranker.predict(new_test_features)
    np.save('/data/Sepehr-data/results_online_'+str(k_n)+'/online_test_pred'+str(i)+'.npy', test_pred)
    np.save('/data/Sepehr-data/results_online_'+str(k_n)+'/online_test_relv'+str(i)+'.npy', new_test_relevances)
    from sklearn.metrics import ndcg_score
    
    new_test_relevances = new_test_relevances.reshape((1, -1))
    test_pred = test_pred.reshape((1, -1))
    #print(test_pred)
    if(len(test_pred[0]) != 1):
        ncdg_res = ndcg_score(new_test_relevances, test_pred, k=10)
        ncdg_cum.append(ncdg_res)
    else:
        print("No NCDG...")
    print("NCDG@10: ", np.mean(ncdg_cum))
    print("Test item No: "+ str(i)+ " of "+ str(len(qid_avg_test)))
    
    

In [ ]:
ranker.evals_result_

In [ ]:
for i in range(0, len(qid_avg_test)):
    test_data_ids = np.where(qid_all_test == qid_avg_test[i])[0]
    new_test_qids = []
    new_test_features = []
    new_test_relevances = []
    for j in range(0, len(test_data_ids)):
            new_test_qids.append(qid_all_test[test_data_ids[j]])
            new_test_features.append(features_all_test[test_data_ids[j]])
            new_test_relevances.append(relevance_all_test[test_data_ids[j]])
    
    new_test_qids = np.array(new_test_qids)
    new_test_features = np.array(new_test_features)
    new_test_relevances = np.array(new_test_relevances)
    
    new_test_relevances = new_test_relevances.reshape(-1)
    new_test_qids = new_test_qids.reshape(-1)
    
    
    from sklearn.metrics import ndcg_score
    
    new_test_relevances = new_test_relevances.reshape((1, -1))
    test_pred = np.load('/data/Sepehr-data/results_online_'+k_n'/online_test_pred'+str(i)+'.npy')
    test_pred = test_pred.reshape((1, -1))
    
    #print(test_pred)
    if(len(test_pred[0]) != 1):
        ncdg_res = ndcg_score(new_test_relevances, test_pred, k=1)
        ncdg_cum.append(ncdg_res)
    else:
        print("No NCDG...")
    print("NCDG@10: ", np.mean(ncdg_cum))
    print("Test item No: "+ str(i)+ " of "+ str(len(qid_avg_test)))
    